使用2014年整个9月份数据作为测试数据，用来验证模型效果。编写Python将数据p的特征进行转换，并对全体数据进行标准化处理

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pickle
import keras
import warnings
warnings.filterwarnings("ignore")

data = pd.read_csv("data/p.csv")
parts = 14
this_one = data.iloc[parts:]
bak_index = this_one.index
for k in range(1, parts + 1):
    last_one = data.iloc[(parts - k):(this_one.shape[0] - k + parts)]
    this_one.set_index(last_one.index, drop=True, inplace=True)
    this_one = this_one.join(last_one, lsuffix="", rsuffix="_p" + str(k))

this_one.set_index(bak_index, drop=True, inplace=True)
this_one = this_one.fillna(0)
t0 = this_one.iloc[:, 0:96]
t0_min = t0.apply(lambda x: np.min(x), axis=0).values
t0_ptp = t0.apply(lambda x: np.ptp(x), axis=0).values
this_one = this_one.apply(lambda x: (x - np.min(x)) / np.ptp(x), axis=0)

Using TensorFlow backend.


这里对负荷数据的标准化参数进行存储，后继对预测数据进行量纲还原。进一步对数据进行分区，得到训练数据和测试数据

In [2]:
test_data = this_one.iloc[-30:]
train_data = this_one.iloc[:-30]
train_y_df = train_data.iloc[:, 0:96]
train_y = np.array(train_y_df)
train_x_df = train_data.iloc[:, 96:]
train_x = np.array(train_x_df)

test_y_df = test_data.iloc[:, 0:96]
test_y = np.array(test_y_df)
test_x_df = test_data.iloc[:, 96:]
test_x = np.array(test_x_df)
test_y_real = t0.iloc[-30:]

with open('data/train_x.pkl', 'wb') as f:
    pickle.dump(train_x, f)
    
with open('data/train_y.pkl', 'wb') as f:
    pickle.dump(train_y, f)
    
with open('data/test_x.pkl', 'wb') as f:
    pickle.dump(test_x, f)
    
with open('data/test_y_real.pkl', 'wb') as f:
    pickle.dump(test_y_real, f)
    
with open('data/t0_ptp.pkl', 'wb') as f:
    pickle.dump(t0_ptp, f)

with open('data/t0_min.pkl', 'wb') as f:
    pickle.dump(t0_min, f)